# Unit 12 - Tales from the Crypto

---


## 1. Sentiment Analysis

Use the [newsapi](https://newsapi.org/) to pull the latest news articles for Bitcoin and Ethereum and create a DataFrame of sentiment scores for each coin.

Use descriptive statistics to answer the following questions:
1. Which coin had the highest mean positive score?
2. Which coin had the highest negative score?
3. Which coin had the highest positive score?

In [ ]:
# Initial imports
import os
import pandas as pd
from dotenv import load_dotenv
import nltk as nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
import nltk
nltk.download()
%matplotlib inline

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\jackc\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


In [ ]:
# Read your api key environment variable
load_dotenv()

api_key = os.getenv("newsAPI")
type(api_key)

In [ ]:
# Create a newsapi client
from newsapi import NewsApiClient
newsapi = NewsApiClient(api_key=api_key)

In [ ]:
# Fetch the Bitcoin news articles
bitcoin_news_articles = newsapi.get_everything(q='bitcoin', language="en", sort_by="relevancy")
bitcoin_news_articles

In [ ]:
# Fetch the Ethereum news articles
ethereum_news_articles = newsapi.get_everything(q='ethereum', language="en", sort_by="relevancy")
ethereum_news_articles

In [ ]:
# Create the Bitcoin sentiment scores DataFrame
bitcoin_sentiments = []

for article in bitcoin_news_articles["articles"]:
    try:
        text = article["content"]
        date = article["publishedAt"][:10]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        
        bitcoin_sentiments.append({
            "text": text,
            "date": date,
            "compound": compound,
            "positive": pos,
            "negative": neg,
            "neutral": neu
            
        })
        
    except AttributeError:
        pass

bitcoin_df = pd.DataFrame(bitcoin_sentiments)
cols = ["date", "text", "compound", "positive", "negative", "neutral"]
bitcoin_df = bitcoin_df[cols]

bitcoin_df.head()

In [ ]:
# Create the Ethereum sentiment scores DataFrame
ethereum_sentiments = []

for article in ethereum_news_articles["articles"]:
    try:
        text = article["content"]
        date = article["publishedAt"][:10]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        
        ethereum_sentiments.append({
            "text": text,
            "date": date,
            "compound": compound,
            "positive": pos,
            "negative": neg,
            "neutral": neu
            
        })
        
    except AttributeError:
        pass

ethereum_df = pd.DataFrame(ethereum_sentiments)
cols = ["date", "text", "compound", "positive", "negative", "neutral"]
ethereum_df = ethereum_df[cols]

ethereum_df.head()

In [ ]:
# Describe the Bitcoin Sentiment
bitcoin_df.describe()  

In [ ]:
# Describe the Ethereum Sentiment
ethereum_df.describe()

### Questions:

Q: Which coin had the highest mean positive score?

A: 

Q: Which coin had the highest compound score?

A: 

Q. Which coin had the highest positive score?

A: 

---

## 2. Natural Language Processing
---
###   Tokenizer

In this section, you will use NLTK and Python to tokenize the text for each coin. Be sure to:
1. Lowercase each word.
2. Remove Punctuation.
3. Remove Stopwords.

In [ ]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from string import punctuation
import re

In [ ]:
# Instantiate the lemmatizer
lemmatizer = WordNetLemmatizer()

# Create a list of stopwords
sw = set(stopwords.words('english'))

# Expand the default stopwords list if necessary
sw_addons = {'said', 'sent', 'found', 'including', 'today', 'announced', 'week', 'basically', 'also'}

In [ ]:
# Complete the tokenizer function
def clean_text(article):
    
    # Create a list of the words
    sw = set(stopwords.words('english'))

    # Convert the words to lowercase
    regex = re.compile("[^a-zA-Z ]")

    # Remove the punctuation
    re_clean = regex.sub('', article)

    # Remove the stop words
    words = word_tokenize(re_clean)

    # Lemmatize Words into root words
    lem = [lemmatizer.lemmatize(word) for word in words]
    # .union combines all items from two sets (AND EXCLUDES DUPLICATES)
    output = [word.lower() for word in lem if word.lower() not in sw.union(sw_addons)]
    
    # Return the final list
    return output
   

In [ ]:
# Create a new tokens column for Bitcoin
bitcoin_df['tokens'] = bitcoin_df['text'].apply(word_tokenize)
bitcoin_df.head()

In [ ]:
# Create a new tokens column for Ethereum
ethereum_df['tokens'] = ethereum_df['text'].apply(word_tokenize)
ethereum_df.head()

---

### NGrams and Frequency Analysis

In this section you will look at the ngrams and word frequency for each coin. 

1. Use NLTK to produce the n-grams for N = 2. 
2. List the top 10 words for each coin. 

In [ ]:
from collections import Counter
from nltk import ngrams

In [ ]:
# Generate the Bitcoin N-grams where N=2
def word_counter(corpus): 
    # Combine all articles in corpus into one large string
    big_string = ' '.join(corpus.text)
    processed = clean_text(big_string)
    bigrams = ngrams(processed, n=2)
    top_10 = dict(Counter(bigrams).most_common(10))
    return pd.DataFrame(list(top_10.items()), columns=['bigram', 'count'])

word_counter(bitcoin_df)

In [ ]:
# Generate the Ethereum N-grams where N=2
word_counter(ethereum_df)

In [ ]:
# Function token_count generates the top 10 words for a given coin
def token_count(tokens, N=10):
    """Returns the top N tokens from the frequency count"""
    return Counter(tokens).most_common(N)

In [ ]:
# Use token_count to get the top 10 words for Bitcoin
bitcoin_text = ' '.join(bitcoin_df.text)
bitcoin_processed = clean_text(bitcoin_text)
token_count(bitcoin_processed)

In [ ]:
# Use token_count to get the top 10 words for Ethereum
ethereum_text = ' '.join(ethereum_df.text)
ethereum_processed = clean_text(ethereum_text)

token_count(ethereum_processed)

---

### Word Clouds

In this section, you will generate word clouds for each coin to summarize the news for each coin

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [20.0, 10.0]

In [ ]:
# Generate the Bitcoin word cloud
bitcoin_words = ' '.join(bitcoin_processed)

bitcoin_cloud = WordCloud().generate(bitcoin_words)
plt.imshow(bitcoin_cloud)

In [ ]:
# Generate the Ethereum word cloud
ethereum_words = ' '.join(ethereum_processed)

ethereum_cloud = WordCloud().generate(ethereum_words)
plt.imshow(ethereum_cloud)

---
## 3. Named Entity Recognition

In this section, you will build a named entity recognition model for both Bitcoin and Ethereum, then visualize the tags using SpaCy.

In [ ]:
import spacy
from spacy import displacy

In [ ]:
# Download the language model for SpaCy
# !python -m spacy download en_core_web_sm

In [ ]:
# Load the spaCy model
nlp = spacy.load('en_core_web_sm')

---
### Bitcoin NER

In [ ]:
# Concatenate all of the Bitcoin text together
bitcoin_text = ' '.join(bitcoin_df.text)

In [ ]:
# Run the NER processor on all of the text
bitcoin_ner = nlp(bitcoin_text)

# Add a title to the document
bitcoin_ner.user_data["title"] = "Bitcoin NER"

In [ ]:
# Render the visualization
displacy.render(btc_ner, style='ent')

In [ ]:
# List all Entities
entity = []
label = []

for ent in btc_ner.ents:
    entity.append(ent.text)
    label.append(ent.label_)
    
data = {"Entity": entity, "Label":label}

bitcoin_entities = pd.DataFrame(data)
bitcoin_entities

---

### Ethereum NER

In [ ]:
# Concatenate all of the Ethereum text together
ethereum_text = ' '.join(ethereum_df.text)

In [ ]:
# Run the NER processor on all of the text
ethereum_ner = nlp(ethereum_text)

# Add a title to the document
ethereum_ner.user_data["title"] = "Ethereum NER"

In [ ]:
# Render the visualization
displacy.render(ethereum_ner, style='ent')

In [ ]:
Ethereum NER
A multi-billion dollar cryptocurrency company has apologised to users after its sale of metaverse land sparked a frenzy that temporarily brought down the Ethereum ORG cryptocurrency. Yuga Labs PERSON , the comp… [+3475 chars] When Bored Ape Yacht Club PERSON creators Yuga Labs PERSON announced its Otherside ORG NFT collection would launch on April 30 DATE , it was predicted by many to be the biggest NFT ORG launch ever. Otherside is an upcoming Bore… [+6669 chars] If you ever wanted to buy an NFT ORG based on Ethereum ORG , you would have to pay a transaction fee to register it on the blockchain. Last week DATE , that fee skyrocketed to unprecedented levels. So you might ha… [+7216 chars] Digital ORG transformation is disrupting global businesses. The legal industry must consider digital technologies to keep changing tech trends. Blockchain-based smart contracts are one CARDINAL example of such te… [+8816 chars] Mike Masnick PERSON wrote a good piece on this topic on his Techdirt blog last week DATE . I particularly like this part: First ORDINAL , lets look at the world without any content moderation. A website that has no cont… [+3109 chars] We are excited to bring Transform ORG 2022 back in-person July 19 DATE and virtually July 20 - 28 DATE . Join AI and data leaders for insightful talks and exciting networking opportunities. Register today DATE ! For ove… [+6087 chars] The U.S. Treasury Department ORG has implicated the North Korea GPE -backed Lazarus Group ORG (aka Hidden Cobra PERSON ) in the theft of $540 million MONEY from video game Axie Infinity's GPE Ronin Network ORG last month DATE . On Thursday DATE … [+5464 chars] Did you miss a session from GamesBeat Summit 2022? All sessions are available to stream now. Learn more.  Jam City GPE is betting big on its first ORDINAL blockchain game, Champions Ascension, and today DATE it dis… [+14217 chars] If youre a current cryptocurrency hodler or a potential investor in India GPE , the government wants you to think twice before putting your money in the sector. For cryptocurrency enthusiasts in the coun… [+7600 chars] This past weekend DATE Bored & ORG amp; Hungry ORG , the world’s Bored Ape Yacht Club restaurant opened in Long Beach GPE , California GPE . Ahead of its opening, the unique concept announced via Twitter PRODUCT that it would not o… [+2021 chars] NFTs are undoubtedly not a fad. Since the explosion of these digital tokens, many projects have been developed which have broadened their utility. But while some of these projects have recorded trem… [+20480 chars] What happened  The cryptocurrency market had another bad end to the week DATE on Friday DATE , selling off into what could be a wild weekend DATE . Yuga Labs PERSON will be minting some form of land for its metaverse and t… [+2413 chars] As the competition between blockchains for developers and users picks up, investors have to decide which cryptocurrency is a likely winner in the future. Ethereum( ETH 1.81% PERCENT ) has long had a big lead… [+2983 chars] During market sell-offs, sometimes it can be difficult to decide what to invest in. Do you buy growth stocks on sale? Maybe you want a dividend stock you can count on for passive income. Or maybe you… [+5931 chars] They grow up so fast! The Ethereum ORG ( ETH 0.60% PERCENT ) cryptocurrency was conceived in 2013 DATE , and the first ORDINAL proper transactions took place two years later DATE . The Ethereum ORG blockchain has rolled out 14 CARDINAL importan… [+3620 chars] 2021 DATE was the year DATE of Ethereum ( ETH 0.08% PERCENT ). The proof-of-stake cryptocurrency stormed the charts, gaining 400% PERCENT to become a legitimate challenger to Bitcoin's ( BTC 0.06% PERCENT ) crypto supremacy. The NF ORG … [+3478 chars] Solana(SOL 0.61% PERCENT ) was one CARDINAL of the hottest cryptocurrencies of 2021 DATE , emerging from relative obscurity and evolving into a crypto that Bank of America ORG called "the Visa of the digital asset ecosystem" by… [+2974 chars] Late last month DATE , airBaltic ORG presented the Planies ORG , the airline’s latest entry into the world of non-fungible tokens (NFTs). The Latvian NORP airline has placed high expectations on this collection of ether… [ +6907 NORP chars] If you don't have much of an interest in the crypto or NFT (non-fungible token) space, you may find Gucci PERSON 's decision to accept payments in cryptocurrency to be strange. However, the brand already has… [+1449 chars] Since 2014 DATE , when Vitalik Buterin PERSON helped to create Ethereum ORG ( ETH 0.95% PERCENT ), he has known there was a problem: scalability. In Ethereum GPE 's white paper (the document that explains how the system is suppose… [+4771 chars]
# List all Entities
entity = []
label = []

for ent in eth_ner.ents:
    entity.append(ent.text)
    label.append(ent.label_)
    
data = {"Entity": entity, "Label":label}

ethereum_entities = pd.DataFrame(data)
ethereum_entities

---